In [ ]:
import numpy as np
import pandas as pd
from utils import load_encrypted_xlsx
import os

In [ ]:
registry_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/sos_sah_data/original_data/aSAH_DATA_2009_2023_24122023.xlsx'
registry_timing_verification_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/sos_sah_data/manual_completions/date_verification_with_infarct_df.xlsx'
post_hoc_modifications_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/sos_sah_data/manual_completions/post_hoc_modifications_sos_db.xlsx'
output_dir = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/sos_sah_data/'

In [ ]:
registry_df = load_encrypted_xlsx(registry_data_path, sheet_name='DATA')
registry_timing_verification_df = load_encrypted_xlsx(registry_timing_verification_path)
post_hoc_modifications_df = load_encrypted_xlsx(post_hoc_modifications_path)

In [ ]:
# Join data from ischemia / infarct timing verification
registry_df = registry_df.merge(registry_timing_verification_df[
                                    ['SOS-CENTER-YEAR-NO.', 'Name', 'Date_DCI_ischemia_first_image',
                                     'Time_DCI_ischemia_first_image', 'Date_DCI_infarct_first_image',
                                     'Time_DCI_infarct_first_image', 'DCI_YN_verified']],
                                on=['SOS-CENTER-YEAR-NO.', 'Name'], how='left')
registry_df['DCI_ischemia'] = registry_df['Date_DCI_ischemia_first_image'].notnull()
registry_df['DCI_infarct'] = registry_df['Date_DCI_infarct_first_image'].notnull()

In [ ]:
post_hoc_modifications_df

In [ ]:
row

In [ ]:
import numpy as np

In [ ]:
# apply modifications of post_hoc_modifications_df to registry_df
for i, row in post_hoc_modifications_df.iterrows():
    try:
        assert registry_df.loc[(registry_df['SOS-CENTER-YEAR-NO.'] == row['SOS-CENTER-YEAR-NO.']) 
                        & (registry_df['Name'] == row['Name'])
                        & (registry_df['Date_birth'] == row['Date_birth'])].shape[0] > 0
        registry_df.loc[(registry_df['SOS-CENTER-YEAR-NO.'] == row['SOS-CENTER-YEAR-NO.']) 
                        & (registry_df['Name'] == row['Name'])
                        & (registry_df['Date_birth'] == row['Date_birth']), row['modified_column']] = row['new_value']
    except:
        if type(row['SOS-CENTER-YEAR-NO.']) != str:
            assert registry_df.loc[(registry_df['Name'] == row['Name'])
                        & (registry_df['Date_birth'] == row['Date_birth'])].shape[0] > 0
            registry_df.loc[(registry_df['Name'] == row['Name'])
                        & (registry_df['Date_birth'] == row['Date_birth']), row['modified_column']] = row['new_value']
        # elif type(row['Name']) != str:
        #     assert registry_df.loc[(registry_df['SOS-CENTER-YEAR-NO.'] == row['SOS-CENTER-YEAR-NO.'])
        #                 & (registry_df['Date_birth'] == row['Date_birth'])].shape[0] > 0
        #     registry_df.loc[(registry_df['SOS-CENTER-YEAR-NO.'] == row['SOS-CENTER-YEAR-NO.'])
        #                 & (registry_df['Date_birth'] == row['Date_birth']), row['modified_column']] = row['new_value']
        # elif type(row['Date_birth']) != str:
        #     assert registry_df.loc[(registry_df['SOS-CENTER-YEAR-NO.'] == row['SOS-CENTER-YEAR-NO.'])
        #                 & (registry_df['Name'] == row['Name'])].shape[0] > 0
        #     registry_df.loc[(registry_df['SOS-CENTER-YEAR-NO.'] == row['SOS-CENTER-YEAR-NO.'])
        #                 & (registry_df['Name'] == row['Name']), row['modified_column']] = row['new_value']
        else:
            print(f"Could not find row {row['SOS-CENTER-YEAR-NO.']}, {row['Name']}, {row['Date_birth']}")
            continue
        
    print(f"{row['SOS-CENTER-YEAR-NO.']}, {row['Name']}, {row['Date_birth']}, changed {row['modified_column']} to {row['new_value']} because {row['reason']}")
    
    

In [ ]:
# save registry_df
original_file_name = os.path.basename(registry_data_path)
registry_df.to_excel(os.path.join(output_dir, 'post_hoc_modified_' + original_file_name), index=False)
# modal warning
print('ADD PASSWORD TO THE FILE!')